# Proof of Concept: 
## Default Categories in Three Different Courses

This notebook demonstrates the full pipeline implemented in the package, illustrating a potential use case in the absence of real data.
In this case, it simulates a researcher who is investigating 30 students' usage of chatbots in three different settings: an English, Biology, and Spanish course. 
The framework used to do so is from XXX, and the researcher is looking into the intentions behind the interactions of both the students and the chatbot.

The steps to do so are as follows:
1. A synthetic framework (i.e., an annotated dataset) is first generated.
2. Next, an interaction is simulated between two agents representing a 'student' and a 'tutor'.
3. A BERT classifier is then trained on the synthesized data. 
4. The trained classifier is then applied to the simulated conversation. In this case, both agents' outputs are analyzed.
5. Finally, the classified data is presented in a descriptive format, showcasing the researchers results. 

In [1]:
pip install educhateval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 7.1 MB/s eta 0:00:00
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached fsspec-2024.12.0-py3-none-any.whl.metadata (11 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.5/102.5 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.2/65.2 kB 5.7 MB/s eta 0:00:00
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached certifi-2025.1.31-py3-none-any.whl.metadata (2.5 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 6.3 MB/s eta 0:00:00
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
     ━━━━━━━━━━━━━━━━━━━━━

In [7]:
from educhateval import FrameworkGenerator, DialogueSimulator

In [6]:
generator = FrameworkGenerator(model_name="DeepSeek-R1-Distill-Llama-8B", api_url="http://localhost:1234/v1/completions")
df_4 = generator.generate_framework(
    prompt_path="/Users/dklaupaa/Desktop/chat_wrap_package/src/educhateval/framework_generation/outline_prompts/prompt_default_4types.py", 
    num_samples=5, 
    csv_out="/Users/dklaupaa/Downloads/TEST.csv"
)

Generating for category: Clarification
Generating for category: Small Talk
Generating for category: Question
Generating for category: Statement
Number of duplicates removed: 2
→ Saved CSV: /Users/dklaupaa/Downloads/TEST.csv


In [17]:
simulator = DialogueSimulator(backend="mlx", model_id="mlx-community/Qwen2.5-7B-Instruct-1M-4bit")

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

In [ ]:
simulator_hf = DialogueSimulator(backend="hf", model_id="microsoft/Phi-3.5-mini-instruct")

tokenizer_config.json:   0%|          | 0.00/3.98k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/195 [00:00<?, ?B/s]

Some parameters are on the meta device because they were offloaded to the disk.


In [20]:
seed_message = "Hi, I'm a student seeking assistance with my studies in English." # example input - same as default

df_single = simulator.simulate_dialogue(
    mode="general_task_solving",
    turns=5,
    seed_message_input=seed_message
    #save_csv_path=Path("output/dialogue.csv")
)


--- Starting Dialogue Simulation ---

[Student]: Hi, I'm a student seeking assistance with my studies in English.
[Tutor]: Great to help! What specific English topic or question do you need assistance with?

Turn 2:
[Student]: Could you please provide an example of the type of English problem or question I should be focusing on?
[Tutor]: Sure! How about working on sentence structure, vocabulary, grammar, or perhaps improving essay writing skills?

Turn 3:
[Student]: Got it. Let's focus on improving sentence structure. Can you give me an example of a weak sentence and a corrected version?
[Tutor]: Sure! 

Weak sentence: "The cat sat on the mat."

Improved: "The cat comfortably settled on the neatly placed mat." 

This adds more detail and clarity.

Turn 4:
[Student]: Thank you. How about we focus on identifying the subject and predicate in a sentence? Could you give me a sentence and help me identify its parts?
[Tutor]: Of course! Let's use this sentence: "The dog chased the squirrel."

KeyError: 'role'